In [2]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address        Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.192.4  242.4 KiB  16      70.5%             770d84d1-39e4-478f-991a-62619ebdb874  rack1
UN  192.168.192.3  25.55 KiB  16      65.0%             9611be36-0777-4331-954c-109cd67c8ccb  rack1
UN  192.168.192.2  25.55 KiB  16      64.5%             bb63e485-ad0b-461a-bdff-336281899fd8  rack1



# Part 1: Station Data

In [3]:
# Connect to the Cassandra cluster using this code:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [4]:
# Then write code to do the following:

# drop a weather keyspace if it already exists
cass.execute("DROP KEYSPACE IF EXISTS weather;")

# create a weather keyspace with 3x replication
cass.execute("""
CREATE KEYSPACE weather WITH
replication = {'class': 'SimpleStrategy', 'replication_factor': 3};
""")

In [5]:
#Use the weather keyspace
cass.execute("use weather")

# inside weather, create a station_record type containing two ints: tmin and tmax
cass.execute("""
create type station_record(
    tmin INT,
    tmax INT
)
""")

In [6]:
#Inside weather, create a station table
#The stations table should have four columns: 
#id (text), name (text), date (date), record (weather.station_record):

# id is a partition key and corresponds to a station's ID (like 'USC00470273')
# date is a cluster key, ascending
# name is a static field (because there is only one name per ID). Example: 'UW ARBORETUM - MADISON'
# record is a regular field because there will be many records per station partition.

cass.execute("""
create table stations(
    id TEXT,
    name TEXT static,
    date DATE,
    record weather.station_record,
    PRIMARY KEY ((id), date)
) WITH CLUSTERING ORDER BY (date ASC)
""")

In [7]:
#q1
# What is the Schema of stations?
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, col
import pandas as pd

spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

station_data = spark.read.text("/nb/ghcnd-stations.txt")

#substring(str, pos, len)
#(end - start )+ 1
station_data = station_data.select(
    substring('value', 1, 11).alias('ID'),  
    substring('value', 39, 2).alias('STATE'),
    substring('value', 42, 30).alias('NAME')
)

# printing Dataframe schema to get the column names
#station_data.printSchema()

# visualizing the dataframe
#station_data.show(truncate=False)

wi_station_df = station_data.filter(col('STATE') == 'WI')
wi_station = wi_station_df.collect() 

for row in wi_station:
    wi_insert = cass.prepare(
            f"INSERT INTO weather.stations (id, name) VALUES (?, ?)"
        )
    cass.execute(wi_insert, (row['ID'], row['NAME']))
    
output = pd.DataFrame(cass.execute("SELECT COUNT(*) FROM weather.stations"))
output.iloc[0,0]

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-25114845-255c-4ce6-a168-162fc84e14b3;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

1313

In [9]:
#q2
#what is the name corresponding to station ID US1WIMR0003?
output = pd.DataFrame(cass.execute("""
SELECT name
FROM weather.stations
WHERE id = 'US1WIMR0003'
"""))
output.iloc[0,0]

'AMBERG 1.3 SW                 '

In [10]:
#q3
# what is the token for the USC00470273 station?
output = pd.DataFrame(cass.execute("""
SELECT TOKEN(ID)
FROM weather.stations
WHERE id = 'USC00470273'
"""))

output.iloc[0,0]

-9014250178872933741

In [11]:
#q4

# what is the first vnode token in the ring following the token for USC00470273?
import re #To read the output using regex
from subprocess import check_output

 

result = cass.execute("""
SELECT TOKEN(ID)
FROM weather.stations
WHERE id = 'USC00470273'
""")
#Use check_output to run nodetool ring
output = check_output(["nodetool", "ring"])
tokens = re.findall(r"-?\d{19}", output.decode("utf-8")) #Use regex to find all the token numbers in the output

#Each token in tokens is a string so covert to ints with list comphrehension
tokens = [int(token) for token in tokens]

#Find the next token after the target token
#sort the tokens
tokens.sort()
target_token = result[0][0]
next_token = None
#Loop through the tokens until we find the next bigger token than the current token for USC00470273
for token in tokens:
    if token > target_token:
        next_token = token
        break #Break because we want to find just the next token thats bigger
next_token

/tmp/ipykernel_184/2112290626.py:24: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  target_token = result[0][0]


-8436275424918165244

# Part 2 Weather Data

In [12]:
#Unzip records.zip cell
! unzip -o records.zip
records_df = spark.read.parquet("records.parquet")

#Import statements to answer q5
import grpc
import station_pb2
import station_pb2_grpc
import datetime #Using date time to convert the row.date to %Y-%m-%d format

Archive:  records.zip
  inflating: records.parquet/part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/._SUCCESS.crc  
  inflating: records.parquet/part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
  inflating: records.parquet/part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet  
 extracting: records.parquet/.part-00003-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/_SUCCESS  
 extracting: records.parquet/.part-00000-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00001-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  
 extracting: records.parquet/.part-00002-574ab704-2431-4c8b-9d88-6c635a467b99-c000.snappy.parquet.crc  


In [14]:
record_output = records_df.groupBy("station", "date").pivot("element", ["TMIN", "TMAX"]).sum("value")

channel = grpc.insecure_channel(f"localhost:5440")
stub = station_pb2_grpc.StationStub(channel)
    
for row in record_output.collect():
    date_output = datetime.datetime.strptime(row.date, '%Y%m%d') #Convert to YYYY-MM-DD TIME
    date_output_formatted = date_output.strftime("%Y-%m-%d") #Remove the TIME
    response = stub.RecordTemps(station_pb2.RecordTempsRequest(station = row.station, date = date_output_formatted, tmin = int(row.TMIN), tmax = int(row.TMAX)))
    # error = response.error
    # print(error)


In [15]:
#q5
#what is the max temperature ever seen for station USW00014837?
#Request for station "USW00014837
request = station_pb2.StationMaxRequest(station = "USW00014837")

max_response= stub.StationMax(request)
max_response.tmax

356

# Part 3: Spark Analysis

In [16]:
# Create a temporary view in Spark named stations that corresponds to the stations table in Cassandra.
# Hint: you already enabled CassandraSparkExtensions when creating your Spark session, 
#so you can create a Spark DataFrame corresponding to a Cassandra table like this:
stations_df_spark = (spark.read.format("org.apache.spark.sql.cassandra")
.option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")
.option("keyspace", 'weather')
.option("table", 'stations')
.load())

stations_df_spark.createOrReplaceTempView("stations")

In [17]:
#q6
#what tables/views are available in the Spark catalog?
tables_list = spark.catalog.listTables()
#len(tables_list) #Only one table
tables_list

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [18]:
#q7
#what is the average difference between tmax and tmin, 
#for each of the four stations that have temperature records?
from pyspark.sql.functions import col


output_df = spark.sql("""
SELECT *
FROM stations
WHERE record IS NOT NULL
""")

#Separate the record column row objects into their own columns
output_df = (output_df.withColumn('temp_min', col('record')['tmin'])
            .withColumn('temp_max', col('record')['tmax'])
            .withColumn('temp_diff', col('temp_max') - col('temp_min')))

output_df = output_df.drop("temp_min", "temp_max") # Drop the min and max columns

#Convert to pandas, group by id, calculate the mean of temp_diff, convert to dict,
average_temp_diff_dict = output_df.toPandas().groupby("id").mean("temp_diff").to_dict()['temp_diff']
average_temp_diff_dict

{'USR0000WDDG': 102.06849315068493,
 'USW00014837': 105.62739726027397,
 'USW00014839': 89.6986301369863,
 'USW00014898': 102.93698630136986}

24/04/14 17:44:10 WARN ChannelPool: [s0|p6-db-2/192.168.192.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=3f10f974-2036-43e8-868e-ba60c4cab8f1, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713116453606}): failed to send request (java.nio.channels.NotYetConnectedException))
24/04/14 17:44:17 WARN ChannelPool: [s0|p6-db-2/192.168.192.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=3f10f974-2036-43e8-868e-ba60c4cab8f1, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713116453606}): failed to send request (java.nio.channels.NotYetConnectedException))


# Part 4: Disaster Strikes

In [19]:
#Run Docker command 'docker stop p6-db-2' to kill the container

In [21]:
#q8
#what does nodetool status output?

#Use the ! COMMAND to show the output in a cell
!nodetool status

24/04/14 17:44:26 WARN ChannelPool: [s0|p6-db-2/192.168.192.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=3f10f974-2036-43e8-868e-ba60c4cab8f1, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713116453606}): failed to send request (java.nio.channels.NotYetConnectedException))


Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address        Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.192.4  98.03 KiB  16      100.0%            770d84d1-39e4-478f-991a-62619ebdb874  rack1
UN  192.168.192.3  87.73 KiB  16      100.0%            9611be36-0777-4331-954c-109cd67c8ccb  rack1
DN  192.168.192.2  87.74 KiB  16      100.0%            bb63e485-ad0b-461a-bdff-336281899fd8  rack1



In [25]:
#q9
#if you make a StationMax RPC call, what does the error field contain in StationMaxReply reply?
request = station_pb2.StationMaxRequest(station = "USW00014837")

max_response = stub.StationMax(request)
max_response.error

'need 3 replicas, but only have 2'

24/04/14 17:52:14 WARN ChannelPool: [s0|p6-db-2/192.168.192.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=3f10f974-2036-43e8-868e-ba60c4cab8f1, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713116453606}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))


In [24]:
#q10
# if you make a RecordTempsRequest RPC call, what does error contain in the RecordTempsReply reply?

#make up data
request = stub.RecordTemps(station_pb2.RecordTempsRequest(station = "UUUU1111111", date = "2022-01-01", tmin = -1000, tmax = 1000))
request.error # should be ''

''

24/04/14 17:48:16 WARN ChannelPool: [s0|p6-db-2/192.168.192.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=3f10f974-2036-43e8-868e-ba60c4cab8f1, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713116453606}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))
24/04/14 17:49:19 WARN ChannelPool: [s0|p6-db-2/192.168.192.2:9042]  Error while opening new channel (ConnectionInitException: [s0|connecting...] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=3f10f974-2036-43e8-868e-ba60c4cab8f1, APPLICATION_NAME=Spark-Cassandra-Connector-local-1713116453606}): failed to send request (com.datastax.oss.driver.shaded.netty.channel.Stackles